In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [13]:
# Load JSON data into a Pandas DataFrame
data_json = pd.read_json('data/politifact_factcheck_data.json' , lines=True)

data_json.head(10)


,verdict,statement_originator,statement,statement_date,statement_source,factchecker,factcheck_date,factcheck_analysis_link
0,true,Barack Obama,John McCain opposed bankruptcy protections for...,6/11/2008,speech,Adriel Bettelheim,6/16/2008,https://www.politifact.com/factchecks/2008/jun...
1,false,Matt Gaetz,"""Bennie Thompson actively cheer-led riots in t...",6/7/2022,television,Yacob Reyes,6/13/2022,https://www.politifact.com/factchecks/2022/jun...
2,mostly-true,Kelly Ayotte,"Says Maggie Hassan was ""out of state on 30 day...",5/18/2016,news,Clay Wirestone,5/27/2016,https://www.politifact.com/factchecks/2016/may...
3,false,Bloggers,"""BUSTED: CDC Inflated COVID Numbers, Accused o...",2/1/2021,blog,Madison Czopek,2/5/2021,https://www.politifact.com/factchecks/2021/feb...
4,half-true,Bobby Jindal,"""I'm the only (Republican) candidate that has ...",8/30/2015,television,Linda Qiu,8/30/2015,https://www.politifact.com/factchecks/2015/aug...
5,true,S.E. Cupp,"""There are actually only 30 countries that pra...",8/23/2015,news,Will Cabaniss,8/23/2015,https://www.politifact.com/factchecks/2015/aug...
6,false,Michele Bachmann,"""My husband and I have never gotten a penny of...",6/26/2011,news,Robert Farley,7/1/2011,https://www.politifact.com/factchecks/2011/jul...
7,true,George Turner,"""If you go strictly by the numbers, crime is d...",4/4/2011,news,Willoughby Mariano,4/18/2011,https://www.politifact.com/factchecks/2011/apr...
8,mostly-true,Kent Conrad,"""The American people say, don't touch Social S...",1/23/2011,other,Angie Drobnic Holan,1/27/2011,https://www.politifact.com/factchecks/2011/jan...
9,true,Mandela Barnes,"""Since 1978, CEO compensation rose over 1,000%...",5/23/2021,social_media,Madeline Heim,6/2/2021,https://www.politifact.com/factchecks/2021/jun...
